In [ ]:
# kobart 설치 및 환경 설정

# pip install git+https://github.com/SKT-AI/KoBART#egg=kobart
# pip install torchtext==0.8.0
# pip install torch==1.7.0
# pip install pytorch_lightning
# pip install transformers==4.0.0

In [ ]:
pip list

# 패키지 설치


In [ ]:
!pip uninstall torch

Found existing installation: torch 1.10.0+cu111
Uninstalling torch-1.10.0+cu111:
  Would remove:
    /usr/local/bin/convert-caffe2-to-onnx
    /usr/local/bin/convert-onnx-to-caffe2
    /usr/local/bin/torchrun
    /usr/local/lib/python3.7/dist-packages/caffe2/*
    /usr/local/lib/python3.7/dist-packages/torch-1.10.0+cu111.dist-info/*
    /usr/local/lib/python3.7/dist-packages/torch/*
Proceed (y/n)? y
  Successfully uninstalled torch-1.10.0+cu111


In [ ]:
!pip install torch==1.7.0

     |████████████████████████████████| 776.7 MB 4.4 kB/s 
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
torchvision 0.11.1+cu111 requires torch==1.10.0, but you have torch 1.7.0 which is incompatible.
torchtext 0.11.0 requires torch==1.10.0, but you have torch 1.7.0 which is incompatible.


In [ ]:
!pip uninstall torchtext
!pip install torchtext==0.8.0

Found existing installation: torchtext 0.11.0
Uninstalling torchtext-0.11.0:
  Would remove:
    /usr/local/lib/python3.7/dist-packages/torchtext-0.11.0.dist-info/*
    /usr/local/lib/python3.7/dist-packages/torchtext/*
Proceed (y/n)? y
  Successfully uninstalled torchtext-0.11.0
     |████████████████████████████████| 6.9 MB 3.8 MB/s 


In [ ]:
!pip install pytorch_lightning==1.1.0
!pip install transformers==4.0.0

In [ ]:
!pip install git+https://github.com/SKT-AI/KoBART #egg=kobart

In [ ]:
!pip uninstall PyYAML
!pip install PyYAML==5.4.1

In [ ]:
# !pip install streamlit==0.72.0        ## 모델 deployment 웹 구현용

# 마운트 

In [ ]:
# 마운트 및 py 사용 가능하게 하는 코드

from google.colab import drive

drive.mount('/content/drive', force_remount=True)

# enter the foldername in your Google Drive where you have saved the unzipped

%cd drive/My\ Drive
%cp -r $FOLDERNAME ../../

Mounted at /content/drive
/content/drive/My Drive
cp: missing destination file operand after '../../'
Try 'cp --help' for more information.


In [ ]:
import os
os.chdir('/content/drive/MyDrive/Colab Notebooks/컨퍼런스')

# Train 모듈 설정

In [ ]:
# train 모듈

import argparse
import logging
import os
import numpy as np
import pandas as pd
import pytorch_lightning as pl
import torch
from pytorch_lightning import loggers as pl_loggers
from torch.utils.data import DataLoader, Dataset
from dataset import KoBARTSummaryDataset
from transformers import BartForConditionalGeneration, PreTrainedTokenizerFast
from transformers.optimization import AdamW, get_cosine_schedule_with_warmup

from kobart import get_pytorch_kobart_model, get_kobart_tokenizer

tokenizer = PreTrainedTokenizerFast.from_pretrained('gogamza/kobart-base-v1')                    ## 에러 부분 -> 원본 get_bart_tokenizer() 


'''
from transformers import BartForConditionalGeneration, PreTrainedTokenizerFast, BartModel

model = BartModel.from_pretrained('gogamza/kobart-base-v1')
'''



parser = argparse.ArgumentParser(description='KoBART translation')

parser.add_argument('--checkpoint_path',
                    type=str,
                    help='checkpoint path')

logger = logging.getLogger()
logger.setLevel(logging.INFO)


class ArgsBase():
    @staticmethod
    def add_model_specific_args(parent_parser):
        parser = argparse.ArgumentParser(
            parents=[parent_parser], add_help=False)
        parser.add_argument('--train_file',
                            type=str,
                            default='data/train.tsv',            # train data 
                            help='train file')

        parser.add_argument('--test_file',
                            type=str,
                            default='data/test.tsv',             # test data
                            help='test file')

        parser.add_argument('--batch_size',
                            type=int,
                            default=28,
                            help='')

        parser.add_argument('--max_len',
                            type=int,
                            default=512,
                            help='max seq len')
        return parser

class KobartSummaryModule(pl.LightningDataModule):
    def __init__(self, train_file,
                 test_file, tok,
                 max_len=512,
                 batch_size=8,
                 num_workers=5):
        super().__init__()
        self.batch_size = batch_size
        self.max_len = max_len
        self.train_file_path = train_file
        self.test_file_path = test_file
        if tok is None:
            self.tok = tokenizer                                                     ####### tokenizer 교체
        else:
            self.tok = tok
        self.num_workers = num_workers

    @staticmethod
    def add_model_specific_args(parent_parser):
        parser = argparse.ArgumentParser(
            parents=[parent_parser], add_help=False)
        parser.add_argument('--num_workers',
                            type=int,
                            default=5,
                            help='num of worker for dataloader')
        return parser

    # OPTIONAL, called for every GPU/machine (assigning state is OK)
    def setup(self, stage):
        # split dataset
        self.train = KoBARTSummaryDataset(self.train_file_path,
                                 self.tok,
                                 self.max_len)
        self.test = KoBARTSummaryDataset(self.test_file_path,
                                self.tok,
                                self.max_len)

    def train_dataloader(self):
        train = DataLoader(self.train,
                           batch_size=self.batch_size,
                           num_workers=self.num_workers, shuffle=True)
        return train

    def val_dataloader(self):
        val = DataLoader(self.test,
                         batch_size=self.batch_size,
                         num_workers=self.num_workers, shuffle=False)
        return val

    def test_dataloader(self):
        test = DataLoader(self.test,
                          batch_size=self.batch_size,
                          num_workers=self.num_workers, shuffle=False)
        return test




class Base(pl.LightningModule):
    def __init__(self, hparams, **kwargs) -> None:
        super(Base, self).__init__()
        self.hparams = hparams

    @staticmethod
    def add_model_specific_args(parent_parser):
        # add model specific args
        parser = argparse.ArgumentParser(
            parents=[parent_parser], add_help=False)

        parser.add_argument('--batch-size',
                            type=int,
                            default=14,
                            help='batch size for training (default: 96)')

        parser.add_argument('--lr',
                            type=float,
                            default=3e-5,
                            help='The initial learning rate')

        parser.add_argument('--warmup_ratio',
                            type=float,
                            default=0.1,
                            help='warmup ratio')

        parser.add_argument('--model_path',
                            type=str,
                            default=None,
                            help='kobart model path')
        return parser

    def configure_optimizers(self):
        # Prepare optimizer
        param_optimizer = list(self.model.named_parameters())
        no_decay = ['bias', 'LayerNorm.bias', 'LayerNorm.weight']
        optimizer_grouped_parameters = [
            {'params': [p for n, p in param_optimizer if not any(
                nd in n for nd in no_decay)], 'weight_decay': 0.01},
            {'params': [p for n, p in param_optimizer if any(
                nd in n for nd in no_decay)], 'weight_decay': 0.0}
        ]
        optimizer = AdamW(optimizer_grouped_parameters,
                          lr=self.hparams.lr, correct_bias=False)
        
        # warm up lr
        num_workers = (self.hparams.gpus if self.hparams.gpus is not None else 1) * (self.hparams.num_nodes if self.hparams.num_nodes is not None else 1)
        data_len = len(self.train_dataloader().dataset)
        logging.info(f'number of workers {num_workers}, data length {data_len}')
        num_train_steps = int(data_len / (self.hparams.batch_size * num_workers) * self.hparams.max_epochs)
        logging.info(f'num_train_steps : {num_train_steps}')
        num_warmup_steps = int(num_train_steps * self.hparams.warmup_ratio)
        logging.info(f'num_warmup_steps : {num_warmup_steps}')
        scheduler = get_cosine_schedule_with_warmup(
            optimizer,
            num_warmup_steps=num_warmup_steps, num_training_steps=num_train_steps)
        lr_scheduler = {'scheduler': scheduler, 
                        'monitor': 'loss', 'interval': 'step',
                        'frequency': 1}
        return [optimizer], [lr_scheduler]


class KoBARTConditionalGeneration(Base):
    def __init__(self, hparams, **kwargs):
        super(KoBARTConditionalGeneration, self).__init__(hparams, **kwargs)
        self.model = BartForConditionalGeneration.from_pretrained('gogamza/kobart-base-v1')                                             ### get_pytorch_kobart_model() 교체
        self.model.train()
        self.bos_token = '<s>'
        self.eos_token = '</s>'
        self.pad_token_id = 0
        self.tokenizer = tokenizer                                                                           ####### tokenizer 교체

    def forward(self, inputs):
        attention_mask = inputs['input_ids'].ne(self.pad_token_id).float()
        decoder_attention_mask = inputs['decoder_input_ids'].ne(self.pad_token_id).float()
        
        return self.model(input_ids=inputs['input_ids'],
                          attention_mask=attention_mask,
                          decoder_input_ids=inputs['decoder_input_ids'],
                          decoder_attention_mask=decoder_attention_mask,
                          labels=inputs['labels'], return_dict=True)


    def training_step(self, batch, batch_idx):
        outs = self(batch)
        loss = outs.loss
        self.log('train_loss', loss, prog_bar=True)
        return loss

    def validation_step(self, batch, batch_idx):
        outs = self(batch)
        loss = outs['loss']
        return (loss)

    def validation_epoch_end(self, outputs):
        losses = []
        for loss in outputs:
            losses.append(loss)
        self.log('val_loss', torch.stack(losses).mean(), prog_bar=True)


# 데이터 불러오기 및 변환

In [ ]:
'''# tsv 파일로 변환, 1000개 tr, 250개 te
import pandas as pd

df = pd.read_csv('/content/drive/Shareddrives/D&A/방언-표준어 문장 데이터/제주도_방언표준어_데이터_전처리후_ver1.csv')


df.rename(columns={'표준어 문장':'kr', '방언 문장':'en'}, inplace=True)

df.iloc[:1000, :].to_csv('model_input_tr.tsv', index = False, sep = '\t')
df.iloc[1001: 1250, :].to_csv('model_input_te.tsv', index = False, sep = '\t') '''

# Train 실행 코드

In [ ]:
# 실행코드 

parser = Base.add_model_specific_args(parser)
parser = ArgsBase.add_model_specific_args(parser)
parser = KobartSummaryModule.add_model_specific_args(parser)
parser = pl.Trainer.add_argparse_args(parser)
args = parser.parse_args('')                                  ## 업데이트 본  원본: args = parser.parse_args() --> args, unknown = parser.parse_known_args()  or  args = parser.parse_args('')




# ---------------------------------------------------------------------------------------
# 인자조정 --> 하이퍼 파라미터 직접 지정해야함

args.gradient_clip_val  = 1.0
args.max_epochs  = 3
args.default_root_dir = 'logs'
args.gpus = 1
args.batch_size = 4

# ---------------------------------------------------------------------------------------


logging.info(args)


model = KoBARTConditionalGeneration(args)

dm = KobartSummaryModule(args.train_file,
                    args.test_file,
                    None,
                    max_len=args.max_len,
                    batch_size=args.batch_size,
                    num_workers=args.num_workers)


checkpoint_callback = pl.callbacks.ModelCheckpoint(monitor='val_loss',
                                                    dirpath=args.default_root_dir,
                                                    filename='model_chp/{epoch:02d}-{val_loss:.3f}',
                                                    verbose=True,
                                                    save_last=True,
                                                    mode='min',
                                                    save_top_k=-1,
                                                    prefix='kobart_translation')



tb_logger = pl_loggers.TensorBoardLogger(os.path.join(args.default_root_dir, 'tb_logs'))
lr_logger = pl.callbacks.LearningRateMonitor()

trainer = pl.Trainer.from_argparse_args(args, logger=tb_logger,
                                        callbacks=[checkpoint_callback, lr_logger])
trainer.fit(model, dm)

INFO:root:Namespace(accelerator=None, accumulate_grad_batches=1, amp_backend='native', amp_level='O2', auto_lr_find=False, auto_scale_batch_size=False, auto_select_gpus=False, automatic_optimization=None, batch_size=4, benchmark=False, check_val_every_n_epoch=1, checkpoint_callback=True, checkpoint_path=None, default_root_dir='logs', deterministic=False, distributed_backend=None, enable_pl_optimizer=True, fast_dev_run=False, flush_logs_every_n_steps=100, gpus=1, gradient_clip_val=1.0, limit_test_batches=1.0, limit_train_batches=1.0, limit_val_batches=1.0, log_every_n_steps=50, log_gpu_memory=None, logger=True, lr=3e-05, max_epochs=3, max_len=512, max_steps=None, min_epochs=1, min_steps=None, model_path=None, move_metrics_to_cpu=False, num_nodes=1, num_processes=1, num_sanity_val_steps=2, num_workers=5, overfit_batches=0.0, plugins=None, precision=32, prepare_data_per_node=True, process_position=0, profiler=None, progress_bar_refresh_rate=1, reload_dataloaders_every_epoch=False, replace

Validation sanity check: 0it [00:00, ?it/s]

Training: 0it [00:00, ?it/s]

Validating: 0it [00:00, ?it/s]

Epoch 0, global step 249: val_loss reached 2.22294 (best 2.22294), saving model to "/content/drive/My Drive/Colab Notebooks/컨퍼런스/logs/kobart_translation-model_chp/epoch=00-val_loss=2.223.ckpt" as top 1
INFO:lightning:Epoch 0, global step 249: val_loss reached 2.22294 (best 2.22294), saving model to "/content/drive/My Drive/Colab Notebooks/컨퍼런스/logs/kobart_translation-model_chp/epoch=00-val_loss=2.223.ckpt" as top 1
/usr/local/lib/python3.7/dist-packages/torch/optim/lr_scheduler.py:216: UserWarning: Please also save or load the state of the optimizer when saving or loading the scheduler.
  warnings.warn(SAVE_STATE_WARNING, UserWarning)


Validating: 0it [00:00, ?it/s]

Epoch 1, global step 499: val_loss reached 2.02912 (best 2.02912), saving model to "/content/drive/My Drive/Colab Notebooks/컨퍼런스/logs/kobart_translation-model_chp/epoch=01-val_loss=2.029.ckpt" as top 2
INFO:lightning:Epoch 1, global step 499: val_loss reached 2.02912 (best 2.02912), saving model to "/content/drive/My Drive/Colab Notebooks/컨퍼런스/logs/kobart_translation-model_chp/epoch=01-val_loss=2.029.ckpt" as top 2


Validating: 0it [00:00, ?it/s]

Epoch 2, global step 749: val_loss reached 2.19948 (best 2.02912), saving model to "/content/drive/My Drive/Colab Notebooks/컨퍼런스/logs/kobart_translation-model_chp/epoch=02-val_loss=2.199.ckpt" as top 3
INFO:lightning:Epoch 2, global step 749: val_loss reached 2.19948 (best 2.02912), saving model to "/content/drive/My Drive/Colab Notebooks/컨퍼런스/logs/kobart_translation-model_chp/epoch=02-val_loss=2.199.ckpt" as top 3
Saving latest checkpoint...
INFO:lightning:Saving latest checkpoint...


1

# translation binary 파일 추출과정

In [ ]:
import argparse
from transformers.models.bart import BartForConditionalGeneration
import yaml

parser = argparse.ArgumentParser()
parser.add_argument("--hparams", default=None, type=str)
parser.add_argument("--model_binary", default=None, type=str)
parser.add_argument("--output_dir", default='kobart_translation', type=str)

args = parser.parse_args('')                                                                                                                                 ## 업데이트 본  원본: args = parser.parse_args() --> args, unknown = parser.parse_known_args()  or  args = parser.parse_args('')

args.hparams = '/content/drive/MyDrive/Colab Notebooks/컨퍼런스/logs/tb_logs/default/version_4/hparams.yaml'                                                          ## 경로지정 요망 
args.model_binary = '/content/drive/MyDrive/Colab Notebooks/컨퍼런스/logs/kobart_translation-model_chp/epoch=02-val_loss=2.199.ckpt'                                  ## 경로지정 요망

with open(args.hparams) as f:
    hparams = yaml.load(f)
    
inf = KoBARTConditionalGeneration.load_from_checkpoint(args.model_binary, hparams=hparams)

inf.model.save_pretrained(args.output_dir)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:15: YAMLLoadWarning: calling yaml.load() without Loader=... is deprecated, as the default Loader is unsafe. Please read https://msg.pyyaml.org/load for full details.
  from ipykernel import kernelapp as app


# Demo 추출과정

In [ ]:
from transformers import BartForConditionalGeneration, PreTrainedTokenizerFast

def load_model():
    model = BartForConditionalGeneration.from_pretrained('/content/drive/MyDrive/Colab Notebooks/컨퍼런스/kobart_translation')                               ## 경로지정 요망
    # tokenizer = PreTrainedTokenizerFast.from_pretrained('gogamza/kobart-base-v1')                        
    return model

model = load_model()
tokenizer = PreTrainedTokenizerFast.from_pretrained('gogamza/kobart-base-v1')


In [ ]:
text = "처음으로 그러니까 언니네 때는 그렇게 했겠지"                                                     ## 여기에 테스트할 문장 입력

text = text.replace('\n', '')
input_ids = tokenizer.encode(text)
input_ids = torch.tensor(input_ids)
input_ids = input_ids.unsqueeze(0)
output = model.generate(input_ids, eos_token_id=1, max_length=512, num_beams=5)
output = tokenizer.decode(output[0], skip_special_tokens=True)

In [ ]:
print(output)

처음으로 그러니까 언니네 때는 겅 했겠지
